In [1]:
import glob
import time
import numpy as np
np.set_printoptions(suppress=True)

from SmoothSparseWeights_CVXOPT import *

In [2]:
exampleName="cheburashka"
poses_folder="poses-2"
poses_ind=7
clip_k=4
handle_num=11


# exampleName="cow"
# poses_folder="poses-2"
# poses_ind=5
# clip_k=4
# handle_num=15


# exampleName="wolf"
# poses_folder="poses-1"
# poses_ind=2
# clip_k=4
# handle_num=21



weights = {'W_spatial': 0.0}

In [3]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


['./models/cheburashka-all/poses-2/cheburashka-01.obj', './models/cheburashka-all/poses-2/cheburashka-02.obj', './models/cheburashka-all/poses-2/cheburashka-03.obj', './models/cheburashka-all/poses-2/cheburashka-04.obj', './models/cheburashka-all/poses-2/cheburashka-05.obj', './models/cheburashka-all/poses-2/cheburashka-06.obj', './models/cheburashka-all/poses-2/cheburashka-07.obj', './models/cheburashka-all/poses-2/cheburashka-08.obj', './models/cheburashka-all/poses-2/cheburashka-09.obj', './models/cheburashka-all/poses-2/cheburashka-10.obj', './models/cheburashka-all/poses-2/cheburashka-11.obj']
(6669, 44)
(6669, 33)


In [4]:
#### read our output of simplex_hull.py 
results_path="./results_pipeline/"+exampleName+"-"+str(handle_num)+"/result.txt"
endmembers, mixing_weights=load_result(results_path)
### endmembers shape is handle * poses * 12??

# ### this is to make sure the 3*4 or 4*3 error does not happen.
# endmembers=endmembers.reshape((-1, 4, 3))
# endmembers=np.transpose(endmembers, (0,2,1))
# endmembers=endmembers.reshape((len(mixing_weights), -1, 12))


endmembers=endmembers.reshape((len(mixing_weights),-1))
mixing_weights=mixing_weights.T


print (endmembers.shape)
print (mixing_weights.shape)
print (mixing_weights[:3])
## grad_zero_indices is for col-major
mixing_weights_clip, grad_zero_indices=clip_first_k_values(mixing_weights, clip_k)
print (mixing_weights_clip[:3])
print (col_major(mixing_weights_clip)[grad_zero_indices].sum())

(11, 132)
(6669, 11)
[[0.74340239 0.11506757 0.04097593 0.05160312 0.00088833 0.00834796
  0.00363905 0.00922954 0.0266172  0.00001564 0.00021327]
 [0.74232596 0.11532759 0.04167527 0.05171986 0.00088909 0.00840986
  0.00362964 0.00925433 0.02653997 0.0000158  0.00021263]
 [0.74685587 0.1146399  0.03846723 0.05108695 0.00088842 0.00818181
  0.00365574 0.00924161 0.02675569 0.00001549 0.0002113 ]]
[[0.74340239 0.11506757 0.04097593 0.05160312 0.         0.
  0.         0.         0.         0.         0.        ]
 [0.74232596 0.11532759 0.04167527 0.05171986 0.         0.
  0.         0.         0.         0.         0.        ]
 [0.74685587 0.1146399  0.03846723 0.05108695 0.         0.
  0.         0.         0.         0.         0.        ]]
0.0


In [5]:
scale=find_scale(vertices1)
print (scale)

4.224960630474087


In [6]:
recovered_vertices2=recover_vertices(mixing_weights, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

0.2769974449129444


In [7]:
recovered_vertices2=recover_vertices(mixing_weights_clip, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

initials={'x': col_major(mixing_weights_clip.reshape((-1,len(endmembers)))).reshape((-1,1))}
print (initials['x'].shape)

start=time.time()

res=run(mesh0, mesh1_list, weights, endmembers, mixing_weights.ravel(), grad_zero_indices, initials=initials)

res=res.reshape(mixing_weights.shape)
end=time.time()
print("using time: ", end-start)

recovered_vertices2=recover_vertices(res, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

3101.787312222525
(73359, 1)
(6669, 44)
(6669, 33)
6669 11 11
W_spatial 0.0
     pcost       dcost       gap    pres   dres
 0: -5.5451e+04 -1.5044e+05  4e+05  2e+00  1e-14
 1: -5.4688e+04 -9.9552e+04  8e+04  3e-01  1e-14
 2: -5.2811e+04 -8.0798e+04  5e+04  1e-01  9e-15
 3: -5.1865e+04 -6.8277e+04  3e+04  6e-02  6e-15
 4: -5.1527e+04 -6.0842e+04  2e+04  3e-02  5e-15
 5: -5.1472e+04 -6.0051e+04  2e+04  3e-02  5e-15
 6: -5.1370e+04 -5.7435e+04  2e+04  3e-02  5e-15
 7: -5.0381e+04 -4.7021e+04  2e+04  2e-02  5e-15
 8: -4.6497e+04 -3.1541e+04  2e+04  2e-02  5e-15
 9: -4.3283e+04 -1.8770e+04  2e+04  2e-02  5e-15
10: -3.8548e+04 -3.5627e+03  2e+04  2e-02  5e-15
11: -3.6975e+04  1.5849e+04  3e+04  1e-02  5e-15
12: -3.5543e+04  3.4628e+04  3e+04  1e-02  5e-15
13: -3.1714e+04  6.6733e+04  4e+04  1e-02  5e-15
14: -2.6139e+04  1.2154e+05  5e+04  1e-02  5e-15
15: -2.0067e+04  1.9706e+05  7e+04  1e-02  5e-15
16: -1.2020e+04  2.7633e+05  8e+04  1e-02  5e-15
17:  3.8137e+03  3.8498e+05  1e+05  1e-02  

In [8]:
print (res.reshape((-1, len(endmembers)))[:3]) ### should have same zeros as initial input.
print ((col_major(res)[grad_zero_indices]).sum()) ### should be zero
print ((col_major(res)[grad_zero_indices]).max()) ### should be zero

[[ 0.00000002  0.00000001  0.00000003  0.99999994 -0.          0.
   0.          0.          0.          0.         -0.        ]
 [ 0.00000002  0.00000001  0.00000003  0.99999994 -0.          0.
   0.          0.          0.          0.         -0.        ]
 [ 0.00000002  0.00000001  0.00000003  0.99999994 -0.          0.
   0.          0.          0.          0.         -0.        ]]
6.394437007279925e-19
1.2532906109401274e-10


In [9]:
print (((1-res.sum(axis=-1))**2).sum()/len(res))
print (res.min(), res.max())

1.736796635168749e-32
-5.112378641862516e-12 0.9999999529379339


In [10]:
recover_diff= (res.dot(endmembers)-mixing_weights.dot(endmembers))
recover_diff= (recover_diff**2).sum()
print (recover_diff)

219569744.02383474


In [11]:
# #### read gt per vertex transformation 
# groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+"-*.DMAT")
# print(groundtruth_names)

# gt_all=None
# for i in range(len(groundtruth_names)):
#     gt=load_DMAT(groundtruth_names[i])
#     gt=gt.T
#     gt=gt.reshape((-1,4,3))
#     gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
#     if gt_all is None:
#         gt_all=gt
#     else:
#         gt_all=np.hstack((gt_all, gt))
    
# print(gt_all.shape)


In [12]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

['./models/cheburashka-all/cheburashka.DMAT']
(6669, 11)
7.810955556852425e-29


In [13]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(len(groundtruth_names), -1, 12)

gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) ### is this true?

print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

['./models/cheburashka-all/poses-2/cheburashka-01.Tmat', './models/cheburashka-all/poses-2/cheburashka-02.Tmat', './models/cheburashka-all/poses-2/cheburashka-03.Tmat', './models/cheburashka-all/poses-2/cheburashka-04.Tmat', './models/cheburashka-all/poses-2/cheburashka-05.Tmat', './models/cheburashka-all/poses-2/cheburashka-06.Tmat', './models/cheburashka-all/poses-2/cheburashka-07.Tmat', './models/cheburashka-all/poses-2/cheburashka-08.Tmat', './models/cheburashka-all/poses-2/cheburashka-09.Tmat', './models/cheburashka-all/poses-2/cheburashka-10.Tmat', './models/cheburashka-all/poses-2/cheburashka-11.Tmat']
(11, 132)
(11, 11, 12)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.963511  -0.265796   0.0316154  0.147363 ]
 [ 0.2199     0.85336    0.472673  -0.245936 ]
 [-0.152614  -0.448474   0.88067    

In [16]:
### plot handles corresponding to gt mixing weights, recovered mixing weights, and post-processed mixing weights.
###

eps=1e-10

# Vertices=vertices2[:,poses_ind*3:poses_ind*3+3]

## use recovered vertices instead.
recovered_vertices2=recover_vertices(res, vertices1, endmembers).reshape((len(vertices1),-1))
Vertices=recovered_vertices2[:,poses_ind*3:poses_ind*3+3]


gt_handle_position=gt_mixing_weights.T.dot(Vertices)/(gt_mixing_weights.T.sum(axis=-1)[...,np.newaxis]+eps)
recovered_handle_position=mixing_weights.T.dot(Vertices)/(mixing_weights.T.sum(axis=-1)[...,np.newaxis]+eps)
cliped_handle_position=mixing_weights_clip.T.dot(Vertices)/(mixing_weights_clip.T.sum(axis=-1)[...,np.newaxis]+eps)
postprocessed_handle_position=res.T.dot(Vertices)/(res.T.sum(axis=-1)[...,np.newaxis]+eps)


%matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')

ax.scatter(gt_handle_position[:,0],gt_handle_position[:,1], gt_handle_position[:,2], c='k', s=50)
ax.scatter(recovered_handle_position[:,0],recovered_handle_position[:,1], recovered_handle_position[:,2], c='r', s=50)
# ax.scatter(cliped_handle_position[:,0],cliped_handle_position[:,1], cliped_handle_position[:,2], c='g', s=50)
ax.scatter(postprocessed_handle_position[:,0],postprocessed_handle_position[:,1], postprocessed_handle_position[:,2], c='b', s=50)
ax.scatter(Vertices[:,0],Vertices[:,1], Vertices[:,2], color='y', s=1, edgecolors='none')

plt.show()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


Using matplotlib backend: MacOSX
